## Features+Head Starter - infer

https://www.kaggle.com/code/nartaa/features-head-starter/notebook

### Configration

In [1]:
# モジュールの動的import(import先のファイルが更新されたときに追従する)
%load_ext autoreload
%autoreload 2

In [2]:
import os, gc, sys

import librosa
import numpy as np
import polars as pl
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt

print("TensorFlow version =", tf.__version__)

2024-03-26 08:35:16.071651: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 08:35:16.090251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 08:35:16.090276: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 08:35:16.090771: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 08:35:16.093912: I tensorflow/core/platform/cpu_feature_guar

TensorFlow version = 2.15.0


In [3]:
# USE MULTIPLE GPUS
gpus = tf.config.list_physical_devices('GPU')
if len(gpus)<=1:
    strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    print(f'Using {len(gpus)} GPU')
else:
    strategy = tf.distribute.MirroredStrategy()
    print(f'Using {len(gpus)} GPUs')

Using 0 GPU


2024-03-26 08:35:18.806639: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-03-26 08:35:18.806684: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: 77aad6f29ef1
2024-03-26 08:35:18.806695: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: 77aad6f29ef1
2024-03-26 08:35:18.806809: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.161.7
2024-03-26 08:35:18.806833: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.161.7
2024-03-26 08:35:18.806840: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.161.7


In [ ]:
# For Kaggle
# sys.path.append('/kaggle/input/d/komekami/hms-harmful-brain-activity-classification')

In [4]:
from src.cfg.v2 import CFG

In [5]:
# 自動混合精度の設定
# https://cocoinit23.com/tensowflow-automatic-mixed-precision/
if CFG.MIX:
  tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
  print("Mixed precision enabled")
else:
  print("Using full precision")

Mixed precision enabled


In [6]:
TARGETS = ["seizure_vote", "lpd_vote", "gpd_vote", "lrda_vote", "grda_vote", "other_vote"]
META = ["spectrogram_id", "spectrogram_label_offset_seconds", "patient_id", "expert_consensus"]

FEATS = [['Fp1','F7','T3','T5','O1'],
         ['Fp1','F3','C3','P3','O1'],
         ['Fp2','F8','T4','T6','O2'],
         ['Fp2','F4','C4','P4','O2']]
FEATS2 = ['Fp1','T3','C3','O1','Fp2','C4','T4','O2']

### Load test csv

In [14]:
test_df = pd.read_csv(os.path.join(CFG.BASE_PATH, "test.csv")).rename({'spectrogram_id':'spec_id'},axis=1)

test_df.head()

,spec_id,eeg_id,patient_id
0,853520,3911565283,6885


### Load test spectrograms and eegs

In [15]:
%%time
files = os.listdir(os.path.join(CFG.BASE_PATH, "test_spectrograms"))
print(f"There are {len(files)} spectrogram parquets")

specs = {}
for i, v in enumerate(files):
  tmp_df = pd.read_parquet(os.path.join(CFG.BASE_PATH, "test_spectrograms", v))
  name = int(v.split('.')[0])

  specs[name] = tmp_df.iloc[:, 1:].values

print(specs)

There are 1 spectrogram parquets
{853520: array([[14.91, 17.11, 11.66, ...,  0.05,  0.04,  0.05],
       [11.13, 10.95, 10.77, ...,  0.03,  0.03,  0.02],
       [10.88, 10.57,  8.79, ...,  0.05,  0.06,  0.06],
       ...,
       [ 9.61, 13.32,  9.19, ...,  0.39,  0.56,  0.29],
       [ 8.43, 11.84, 13.64, ...,  0.45,  0.45,  0.34],
       [12.33, 11.84,  9.42, ...,  0.46,  0.54,  0.29]], dtype=float32)}
CPU times: user 23.2 ms, sys: 374 µs, total: 23.5 ms
Wall time: 11.5 ms


In [16]:
def spectrogram_from_eeg(parquet_path):
    # Load middle 50sec of EEG series
    eeg_df = pl.read_parquet(parquet_path).to_pandas()
    middle = (len(eeg_df) - 10_000) // 2
    eeg_df = eeg_df.iloc[middle:middle+10_000]

    # variable to hold spectrogram
    img = np.zeros((128, 256, 4), dtype="float32")

    signals = []
    for k in range(4):
        COLS = FEATS[k]

        for kk in range(4):
            # compute pair differences
            x = eeg_df[COLS[kk]].values - eeg_df[COLS[kk+1]].values

            # fill nan
            m = np.nanmean(x)
            if np.isnan(x).mean() < 1:
                x = np.nan_to_num(x, nan=m)
            else:
                x[:] = 0

            signals.append(x)

            # Raw spectrogram
            mel_spec = librosa.feature.melspectrogram(
                y=x,
                sr=200,
                hop_length=len(x)//256,
                n_fft=1024,
                n_mels=128,
                fmin=0,
                fmax=20,
                win_length=128,
            )

            # Log transform
            width = (mel_spec.shape[1] // 32) * 32
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[:, :width]

            # Standardize to -1 to 1
            mel_spec_db = (mel_spec_db+40)/40
            img[:,:,k] += mel_spec_db

        # Average the 4montage differences
        img[:,:,k] /= 4.0

    return img

In [17]:
eegs = {}
for i, v in enumerate(test_df["eeg_id"].unique()):
    img = spectrogram_from_eeg(os.path.join(CFG.BASE_PATH, "test_eegs", f"{v}.parquet"))
    eegs[v] = img

### DataLoader

In [18]:
# For kaggle
# !pip install --no-index --find-links=/kaggle/input/tf-efficientnet-whl-files /kaggle/input/tf-efficientnet-whl-files/efficientnet-1.1.1-py3-none-any.whl

In [19]:
from src.loaders.dataloader import DataLoader

test_loader = DataLoader(
    df=test_df,
    specs=specs,
    eegs=eegs,
    augment=False,
    mode="test",
)

### Infer

In [20]:
from src.models.efficientnet_b0 import EfficientNetB0

model = EfficientNetB0.build_model()

preds = []
for i in range(5):
    print(f"Fold {i+1}")
    if CFG.LOAD_MODELS_FROM_INFER == "":
        model.load_weights(f"EfficientNet_v{CFG.VER}_f{i}.weights.h5")
    else:
        model.load_weights(f"{CFG.LOAD_MODELS_FROM_INFER}/EfficientNet_v{CFG.VER}_f{i}.weights.h5")

    pred = model.predict(test_loader, verbose=1)
    preds.append(pred)
pred = np.mean(preds, axis=0)

print("Test preds shape", pred.shape)


Fold 1


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/kaggle/input/hms-efficientnet-b0/EfficientNet_v2_f0.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
sub = pd.DataFrame({"eeg_id": test_df["eeg_id"].values})
sub[label_columns] = pred
sub.to_csv("submission.csv", index=False)

print("Submission shape", sub.shape)
sub.head()

Submission shape (1, 7)


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.168616,0.029283,0.000258,0.455246,0.018882,0.327716


In [ ]:
# Check to confirm predictions sum to one
sub.iloc[:, -6:].sum(axis=1)

0    1.0
dtype: float32